In [4]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data, get_sensitive_directions_and_projection_matrix
from sklearn import linear_model

ImportError: Traceback (most recent call last):
  File "C:\Users\maity\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\maity\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\maity\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.752.0_x64__qbz5n2kfra8p

In [1]:
seed = 1
X_train, X_test, y_train, y_test, X_gender_train, X_gender_test, y_gender_train, y_gender_test, dataset_orig_train, dataset_orig_test = preprocess_adult_data(seed = seed)

NameError: name 'preprocess_adult_data' is not defined

In [3]:
X_train

array([[ 1.02285308, -0.04535909, -0.14741329, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.34057873,  1.12591555, -0.14741329, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.34057873,  1.12591555, -0.14741329, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-1.32720302, -0.04535909, -0.14741329, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.94704482,  1.51634043, -0.14741329, ...,  0.        ,
         1.        ,  1.        ],
       [ 1.09866134, -1.99748348, -0.14741329, ...,  0.        ,
         1.        ,  0.        ]])

In [4]:
y_train

array([0., 1., 0., ..., 0., 1., 0.])

In [5]:
y_gender_test

array([1., 1., 1., ..., 1., 0., 0.])

In [6]:
y_test

array([0., 0., 0., ..., 0., 0., 0.])

In [8]:
sensitive_regression = linear_model.LinearRegression(fit_intercept = True)
sensitive_regression.fit(X_gender_train, y_gender_train)
intercept = np.reshape(sensitive_regression.intercept_, (-1,1))


In [9]:
intercept

array([[0.49470783]])

In [10]:
sensitive_regression.coef_

array([-1.45122249e-03,  8.97351853e-03,  5.30371924e-03,  6.54453195e-03,
        3.08378326e-02,  3.30152841e-02, -4.63580179e-02, -2.21780520e-02,
        3.84580924e-02,  1.46502946e-02, -1.79831668e-02,  3.95565592e-04,
       -6.66356168e-03, -4.05661497e-02,  2.95768297e-02,  7.63609017e-02,
        8.75296898e-02,  1.41781647e-02, -1.60415874e-01, -2.20936233e-01,
        2.08546023e-01,  1.41588345e-01, -4.94191370e-02,  1.32619252e-01,
        1.99408337e-01,  3.67427914e-02, -1.01265760e-01, -3.56693477e-01,
       -7.85510986e-02,  8.54324351e-02, -7.09926474e-02, -6.39335621e-02,
        1.37454732e-01,  4.50256061e-01,  4.65622443e-02,  5.70139763e-02,
        5.11480100e-02, -1.84734845e-01, -4.20245446e-01,  1.77095461e-02])

In [11]:
X_train.shape

(36177, 41)

In [18]:
a = X_train[:, 40]

In [19]:
a

array([1., 0., 0., ..., 0., 1., 0.])

In [20]:
y_gender_train

array([0., 0., 0., ..., 0., 1., 1.])

In [21]:
dataset_orig_train

               instance weights  features                             \
                                                                       
                                      age education-num capital-gain   
instance names                                                         
42106                       1.0  1.022853     -0.045359    -0.147413   
31714                       1.0  0.340579      1.125916    -0.147413   
10772                       1.0  0.340579      1.125916    -0.147413   
15825                       1.0 -0.493312     -0.826209    -0.147413   
40653                       1.0 -0.493312     -0.826209    -0.147413   
...                         ...       ...           ...          ...   
37008                       1.0 -0.644929     -0.045359    -0.147413   
43443                       1.0 -0.948162     -0.826209    -0.147413   
20022                       1.0 -1.327203     -0.045359    -0.147413   
43753                       1.0  0.947045      1.516340    -0.14

In [29]:
x_unprotected_train, x_protected_train = dataset_orig_train.features[:, :39], dataset_orig_train.features[:, 39:]
x_unprotected_test, x_protected_test = dataset_orig_test.features[:, :39], dataset_orig_test.features[:, 39:]

In [35]:
protected_regression = linear_model.LinearRegression(fit_intercept = False)
protected_regression.fit(x_unprotected_train, x_protected_train)
sensetive_directions = protected_regression.coef_